In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from pprint import pprint

from FSSLibrary import FFTBasic as fft
import skyboxdatapy as skb

## Function to plot and format spectrum 

In [ ]:
def getSpec(wg, fs, ax, xlim, lbl, fRef=1):    
    fHalf, fAmp, fS = skb.spec.get_single_sided_spectrum(wg, fs)

    
    ax.plot(fHalf/fRef, fAmp, label=lbl)
    ax.set_xlabel("Frequency f/fRef")
    ax.set_ylabel("Amplitude (mm)")
    ax.set_title("Single-Sided FFT")
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.legend()
    ax.set_xlim(xlim)
    # plt.show()

    return fHalf, fAmp, fS


## Load the case

In [ ]:
data_folder = "/home/shagun/Acads/SkyBox/ExpData/data_nosync"
testName = "Test470"

file = skb.io.find_unique_file(data_folder, testName, ext="mat")
print(f"Processing file: {file}")

# loaded_mat = skb.io.load_hdf5_mat(file)
loaded_mat = skb.io.load_case(file)

fSampling = loaded_mat['TestProperties']['fSampling']

## Test Properties

In [ ]:
skb.utils.print_test_properties(loaded_mat)

## List all headers in the dict

In [ ]:
skb.utils.print_all_headers(loaded_mat)            

## Extract Default Dataset

In [ ]:
ds = loaded_mat["DefaultData"]

# Sometimes need to skip initial data.
# Skip first 10 indexes
ds = ds.isel(Time=slice(10, None))


skb.utils.print_top_headers(ds)

## Set zero for signals

In [ ]:
ds = skb.postprocess.set_all_probe_tare(ds, start_time=0, end_time=0.5)

pprint(ds.attrs['tare_values'])

## Plot signals and spectrum

In [ ]:
xmin = 0
xmax = 180

dsub = ds.sel(Time=slice(30, 60))

# Plot
plt.figure(figsize=(16,9))
# plt.plot(ds['Time'], ds['WG01'], linestyle='-', label="WG01")
ds['WG01'].plot(linestyle='-', label=f"{testName} WG01")
dsub['WG01'].plot(color='red', linestyle='-', label="Selected Segment")
plt.xlabel("Time")
plt.ylabel("Value")
# plt.yticks(range(0,150,2))
plt.xlim(xmin, xmax)
# plt.ylim(ymin, ymax)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.show()

pass

In [ ]:
# Plot spectrum
fig, ax = plt.subplots(figsize=(16,9))
xlim = (0, 10)
getSpec(dsub['WG01'], fSampling, ax, xlim, "WG01")

pass